In [143]:
import json
from pprint import pprint
from json import JSONDecoder, JSONDecodeError
import re
import pickle

In [144]:
def write(ori,com,_bin,name,first,last):
    temp = name+".ori"
    with open(temp,'w') as f:
        for i in range(first,last):
            f.write(ori[i])
            f.write("\n")
    temp=name+".com"
    with open(temp,'w') as f:
        for i in range(first,last):
            f.write(com[i])
            f.write("\n")
    temp = name+".bin"
    with open(temp,'w') as f:
        for i in range(first,last):
            for j in _bin[i]:
                f.write(str(j))
            f.write("\n")
    

In [145]:
NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            raise
        yield obj

In [146]:
with open("compression_data.json","r") as f:
    data = f.read()

In [147]:
ori_sentence = []
com_sentence = []
bin_sentence = []
count = 0
train_size = 0
valid_size = 0
test_size = 0
for obj in decode_stacked(data):
    count+=1
    index_Start = obj['source_tree']['node'][1]['word'][0]['id']
    index_End = obj['source_tree']['node'][-1]['word'][0]['id']
    if index_End-index_Start+1 >= 50:
        continue
    sentence = obj['graph']['sentence']
    if sentence[-1] != '.':
        sentence+='.'
    ori_sentence.append(sentence)
    sentence = obj['compression_untransformed']['text']
    if sentence[-1] != '.':
        sentence+='.'
    com_sentence.append(sentence)
    temp = [ 0  for i in range(index_End-index_Start+1)]
    for i in obj['compression_untransformed']['edge']:
        temp[i['child_id']-index_Start]=1
    bin_sentence.append(temp)
    if count == 8000:
        train_size = len(ori_sentence)
    elif count == 9000:
        valid_size = len(ori_sentence)
    elif count == 10000:
        test_size = len(ori_sentence)
write(ori_sentence,com_sentence,bin_sentence,"train",0,train_size)
write(ori_sentence,com_sentence,bin_sentence,"valid",train_size,valid_size)
write(ori_sentence,com_sentence,bin_sentence,"test",valid_size,test_size)

    